In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/updatetraintest/test_set.csv
/kaggle/input/updatetraintest/train_set.csv


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install transformers

In [3]:
pip install sentencepiece

In [4]:
import pandas as pd
import torch

from torch.utils.data import Dataset, DataLoader
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW,AutoTokenizer

In [5]:
from sklearn.model_selection import KFold

In [6]:
df=pd.read_csv("/content/drive/MyDrive/Math_new_simple_dataset/csv_converted/outputmultiply.csv")

In [7]:

# Define a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, text_list, equation_list, tokenizer, max_length=256):
        self.text_list = text_list
        self.equation_list = equation_list
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.text_list)

    def __getitem__(self, idx):
        text = str(self.text_list[idx])
        equation = str(self.equation_list[idx])

        inputs = self.tokenizer(
            f"text: {text}",
            return_tensors="pt",
            max_length=self.max_length,
            truncation=True,
            padding="max_length"
        )

        labels = self.tokenizer(
            f"equation: {equation}",
            return_tensors="pt",
            max_length=self.max_length,
            truncation=True,
            padding="max_length"
        )

        return {
            "input_ids": inputs["input_ids"].squeeze(),
            "attention_mask": inputs["attention_mask"].squeeze(),
            "labels": labels["input_ids"].squeeze()
        }


In [8]:

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load model directly

model = AutoModelForSeq2SeqLM.from_pretrained("sammanamgain/FInal_Final_model_Math_large_model")



tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large",model_max_length=256)

# Set device to CPU
device = torch.device("cuda")
model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/818 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=1024, out_features=2816, bias=False)
              (wi_1): Linear(in_features=1024, out_features=2816, bias=False)
       

In [9]:
training_loss=[]
validation_loss=[]

In [10]:
dataloader = DataLoader(dataset=df, batch_size=2, shuffle=True)

In [ ]:
# num_epochs = 3  # Adjust as needed
# for epoch in range(num_epochs):
#     total_loss = 0.0
#     for batch in dataloader:
#         input_ids = batch["input_ids"].to(device)
#         attention_mask = batch["attention_mask"].to(device)
#         labels = batch["labels"].to(device)

#         optimizer.zero_grad()
#         outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs.loss
#         loss.backward()
#         optimizer.step()

#         total_loss += loss.item()

#     average_loss = total_loss / len(dataloader)
#     print(f"Epoch {epoch + 1}/{num_epochs} - Average Loss: {average_loss:.4f}")


In [13]:
df=df[:500]

In [16]:
df.count()

(-29)/(-87)*(-18)/(-10)    500
3/5                        500
dtype: int64

In [20]:
df.iloc[:,0]

0                       What is (-5)/20*-16*2/(-4)?
1       What is the value of (-2*4/(-72))/(1/(-2))?
2                     What is 51/85*((-5)/(-18))/1?
3                         Evaluate (-28)/21*(-9)/2.
4      What is the value of (4/(-1))/(-40)*(-48)/6?
                           ...                     
495                What is (17/(2295/6))/(9/(-15))?
496                     Calculate 2/2*(-3)/(-30)*4.
497       Calculate (((-440)/(-22))/140)/((-6)/35).
498                             1*(-25)/15*30/(-10)
499             What is ((-16)/14)/(15/(420/(-8)))?
Name: (-29)/(-87)*(-18)/(-10), Length: 500, dtype: object

In [22]:
num_epochs = 2  # Adjust as needed
k_folds = 5
kf = KFold(n_splits=k_folds)

fold_losses = []

for fold, (train_index, val_index) in enumerate(kf.split(df)):
    print(f"Fold {fold + 1}/{k_folds}")

    # Create train and validation datasets
    train_data = df.iloc[train_index]
    val_data = df.iloc[val_index]

    train_dataset = CustomDataset(
        text_list=train_data.iloc[:,0].tolist(),
        equation_list=train_data.iloc[:,1].tolist(),
        tokenizer=tokenizer
    )

    val_dataset = CustomDataset(
        text_list=val_data.iloc[:,0].tolist(),
        equation_list=val_data.iloc[:,1].tolist(),
        tokenizer=tokenizer
    )

    # Create DataLoader for training and validation
    train_dataloader = DataLoader(dataset=train_dataset, batch_size=2, shuffle=True)
    val_dataloader = DataLoader(dataset=val_dataset, batch_size=2, shuffle=False)

    # Initialize optimizer
    optimizer = AdamW(model.parameters(), lr=5e-5)

    # Training loop
    for epoch in range(num_epochs):
        total_loss = 0.0
        model.train()
        for batch in train_dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        average_loss = total_loss / len(train_dataloader)
        training_loss.append(average_loss)
        print(f"Epoch {epoch + 1}/{num_epochs} - Train Loss: {average_loss:.4f}")

    #Validation loop
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch in val_dataloader:
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                labels = batch["labels"].to(device)

                outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                val_loss += loss.item()

        average_val_loss = val_loss / len(val_dataloader)
        validation_loss.append(average_val_loss)
        print(f"Validation Loss: {average_val_loss:.4f}")
        fold_losses.append(average_val_loss)
    break

Fold 1/5


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB. GPU 0 has a total capacty of 14.75 GiB of which 63.06 MiB is free. Process 7425 has 14.68 GiB memory in use. Of the allocated memory 14.48 GiB is allocated by PyTorch, and 76.05 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
training_loss


[0.008799188069901152, 0.006470891329486637]

In [ ]:
validation_loss

[0.010236366994767805, 0.010578015445861311]

In [ ]:
model.save_pretrained("/kaggle/working/epochs")